# Example 8: FX carry strategy

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/8_FX_Carry_Strategy.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Get a FRED API key
This notebook uses the FREDAPI library and requires you to have an API key for Federal Reserve Economic Data. Get one [here](https://fred.stlouisfed.org/docs/api/api_key.html).

### 3. Enter your Sigtech and FRED API keys
After pasting in your API keys, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API keys, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install FRED API and our Python SDK
%pip install sigtech 
%pip install fredapi

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

# Define your FRED API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# FRED_API_KEY with the API key you have generated. e.g. os.environ["FRED_API_KEY"]="abcdefghijklmnopqrstuvwxyz123456")
from fredapi import Fred
fred = Fred(api_key=os.environ["FRED_API_KEY"])

# Note - if this cell does not run in Colab, delete the line fred = Fred(api_key=os.environ["FRED_API_KEY"]) and replace it with:
# os.environ["FRED"] = "FRED_API_KEY"
# fred = Fred(api_key=os.environ["FRED"])

### 4. Set up your Colab environment

In [ ]:
# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set any parameters 
plt.rcParams['figure.figsize'] = [16, 8]

### 5. Create a session

After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to an FX carry strategy

An FX carry strategy aims to capitalize on the difference in interest rates between two currencies. The strategy involves borrowing in a currency with a low-interest rate and investing in a currency with a high-interest rate, thereby earning the "carry", i.e. the difference between the two rates.

## Our strategy
- Fetch historical federal funds rate data to use as a benchmark interest rate for the U.S. dollar (USD).
- Get the historical interest rates of various currencies and compare them to the benchmark USD interest rate. 
- Calculate the carry ratio for each pair and uses it to generate trading signals.
- Create a trading strategy using these signals and plot its performance. 

## 1. Get historical data
First, we will use the FRED API to retrieve the historical federal funds rate.

In [ ]:
fedfunds = fred.get_series('FEDFUNDS').loc[dtm.date(2010,1,10):]

In [ ]:
fedfunds.plot()

Next, we will create a universe of the currencies whose interest rates we want to compare to the federal funds rate and map them to their code in the FRED API. 

In [ ]:
universe_mapping = {
    'AD' : 'IR3TIB01AUM156N', # AUDUSD
    'BP' : 'IR3TIB01GBM156N', # GBPUSD
    'CD' : 'IR3TIB01CAM156N', # CADUSD
    'EC' : 'IR3TIB01EZM156N', # EURUSD
    'NV' : 'IR3TIB01NZM156N', # NZDUSD
    'SF' : 'IR3TIB01NZM156N', # CHFUSD
}

## 2. Calculate the carry ratio and generate trading signals
The carry ratio is calculated by subtracting the Federal Funds Rate from the interest rate of the other currency in the pair (which is fetched using the FRED API). This differential effectively tells you how much more (or less) you could earn by holding one currency over the other.

In [ ]:
carry_ratio = pd.DataFrame({x: (fred.get_series(y).loc[dtm.date(2010,1,10):] - fedfunds) for x, y in universe_mapping.items()})

In [ ]:
carry_ratio

The interest rate differential (carry ratio) is then used to generate trading signals. By incorporating the Federal Funds Rate, the strategy can aim for a "dollar-neutral" portfolio, where the sum of all USD exposures is zero. This helps in isolating the carry trade as the primary source of profit or loss, rather than directional moves in the USD itself.

In [ ]:
def long_short_dollar_neutral(ts: pd.DataFrame, proportion: float) -> pd.DataFrame:
    """
    This function creates a long-short dollar-neutral portfolio based on the input time series of asset signals.
    
    :param ts: DataFrame containing time series data of asset signals.
    :param proportion: Proportion of assets to take long and short positions in.
    :return: DataFrame containing the long-short allocations for each asset.
    """
    
    # Check if the proportion value is within the valid range [0, 1]
    assert 0 <= proportion <= 1, f'proportion must be between 0 and 1, "{proportion}" supplied.'

    # Helper function to calculate long and short positions for each row in the time series
    def calc_signal_on_d(ts_row):
        # Drop NaN values from the row
        ts_row = ts_row.dropna()
        
        # Calculate the number of assets to go long and short on
        n = int(len(ts_row) * proportion)
        
        # Take long positions on the top 'n' assets and short positions on the bottom 'n' assets
        row_output = pd.concat([pd.Series(1, index=ts_row.nlargest(n).index),
                                pd.Series(-1, index=ts_row.nsmallest(n).index)])
        
        # Remove duplicate indices to maintain dollar neutrality
        return row_output.loc[row_output.index.drop_duplicates(keep=False)]

    # Calculate long and short positions for each date and concatenate them into a DataFrame
    df = pd.concat({d: calc_signal_on_d(ts.loc[d]) for d in ts.index}, axis=1, sort=True).T.fillna(0)
    
    # Normalize the allocations to make the portfolio dollar-neutral
    return df.divide(df.abs().sum(axis=1), axis=0)


## 3. Create a signal strategy and view its performance

Firstly, we need to use the `long_short_dollar_neutral` function to generate a DataFrame which can be used as a `signal_input` in our SDK's `SignalStrategy` class. 

In [ ]:
all_signals = long_short_dollar_neutral(carry_ratio, 0.5)

In [ ]:
signal_df = all_signals.dropna()

As part of getting `signal_df` in the correct format, we need to change the names of the column rows to match the names of those instruments in the SigTech API. 

In [ ]:
signal_df.columns = [sig.RollingFutureStrategy(contract_code = code, contract_sector = 'CURNCY').name for code in universe_mapping.keys()]
signal_df

Then, we can create and plot the signal strategy.

In [ ]:
strat = sig.SignalStrategy(
    currency = 'USD',
    start_date = dtm.date(2010,2,10),
    signal_input = signal_df,
    rebalance_frequency = 'EOM' # The signal strategy rebalances at the end of each month. 
)


In [ ]:
strat.history().plot()